# [Saving Scraped Data To SQLite Database With Scrapy Pipelines](https://scrapeops.io/python-scrapy-playbook/scrapy-save-data-sqlite/)

# Scrapy PipelineでスクレイピングされたデータをSQLiteに保存する

プロジェクトの作成

```bash
scrapy startproject sqlite_demo
```

Treeの確認

```bash
tree

.
├── scrapy.cfg
└── sqlite_demo
    ├── __init__.py
    ├── items.py
    ├── middlewares.py
    ├── pipelines.py
    ├── settings.py
    └── spiders
        └── __init__.py
```

Spiderの作成

```bash
scrapy genspider quotes quotes.toscrape.com
```

```python
# spiders/quotes.py
import scrapy
from sqlite_demo.sqlite_demo.items import QuoteItem


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    allowed_domains = ["quotes.toscrape.com"]
    start_urls = ["https://quotes.toscrape.com"]

    def start_requests(self):
        url = "https://quotes.toscrape.com/"
        yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        quote_item = QuoteItem()
        for quote in response.css("div.quote"):
            quote_item["text"] = quote.css("span.text::text").get()
            quote_item["author"] = quote.css("small.author::text").get()
            quote_item["tags"] = quote.css("div.tags a.tag::text").getall()
            yield quote_item
```

items.pyの編集

```python
# items.py
from scrapy.item import Item, Field


class QuoteItem(Item):
    text = Field()
    tags = Field()
    author = Field()
```

## 1. SQLite データベースとテーブルの作成

- `__init__`メソッドの中で、pipelineがspiderによって起動されるたびに、次のような処理を行うように設定する。
  - データベース`demo.db`に接続する。存在しない場合はデータベースを作成する。
  - データベースでSQLコマンドを実行するために使用するカーソルを作成する。
  - データベースにまだ存在しない場合は、`text`、`tags`、`author`のカラムを持つ新しいテーブル`quotes`を作成する。

```python
# pipeline.py
import sqlite3


class SqliteDemoPipeline:

    def __init__(self) -> None:

        # Create/Connect to database
        self.con = sqlite3.connect("demo.db")

        # Create cursor, used to execute commands
        self.cur = self.con.cursor()

        # Create quotes table if none exists
        self.cur.execute("""
            CREATE TABLE IF NOT EXISTS quotes(
                text TEXT,
                tags TEXT,
                author TEXT
            )
        """)

    def process_item(self, item, spider):
        return item
```

## 2. スクレイピングしたものをデータベースに保存する

- Scrapy pipelineの中の`process_item`イベントを使って、スクレイピングしたデータをSQLiteデータベースに保存する。
- `process_item`は、spiderによってitemがスクレイピングされるたびに起動されるので、itemsのデータをデータベースに挿入する`process_item`メソッドを設定する。

```python
# pipeline.py
import sqlite3


class SqliteDemoPipeline:

    def __init__(self) -> None:

        # Create/Connect to database
        self.con = sqlite3.connect("demo.db")

        # Create cursor, used to execute commands
        self.cur = self.con.cursor()

        # Create quotes table if none exists
        self.cur.execute("""
            CREATE TABLE IF NOT EXISTS quotes(
                text TEXT,
                tags TEXT,
                author TEXT
            )
        """)

    def process_item(self, item, spider):

        # Define insert statement
        self.cur.execute("""
            INSERT INTO quotes (text, tags, author) VALUES (?, ?, ?)
        """,
        (
            item["text"],
            str(item["tags"]),
            item["author"]
        )
        )

        # Execute insert of data into database
        self.con.commit()
        return item

```

- SQLのinsert文を定義し、データを与える（注、tagsの値は配列なので文字列化する）。
- `self.con.commit()`コマンドを使って、データを挿入する。

## 3. Item Pipelineの有効化

- Item Pipelineを有効にするために、`settings.py`ファイルにincludeする。

```python
# settings.py
ITEM_PIPELINES = {
    "sqlite_demo.pipelines.SqliteDemoPipeline": 300,
}
```

スクレイピングの実行

```bash
scrapy crawl quotes
```

### 新しいデータのみ保存

- itemがすでにデータベースにあるかどうかを確認してから再度挿入するように、pipelineを再構成する。
- `pipelines.py`ファイルに`SqliteNoDuplicatesPipeline`という新しいpipelineを作成し、
- `process_item`メソッドを変更して、新しいデータのみをデータベースに挿入する。
- データベースで`item['text']`を検索し、それがない場合にのみ新しい項目を挿入する。

```python
# pipelines.py
import sqlite3


class SqliteNoDuplicatesPipeline:

    def __init__(self) -> None:

        # Create/Connect to database
        self.con = sqlite3.connect("demo.db")

        # Create cursor, used to execute commands
        self.cur = self.con.cursor()

        # Create quotes table if none exists
        self.cur.execute("""
            CREATE TABLE IF NOT EXISTS quotes(
                text TEXT,
                tags TEXT,
                author TEXT
            )
        """)

    def process_item(self, item, spider):

        # Check to see if text is already in database
        self.cur.execute(
            "select * from quotes where text = ?",
            (item["text"],)
        )
        result = self.cur.fetchone()

        # If it is in DB, create log message
        if result:
            spider.logger.warn(
                "Item already in database: %s" % item["text"]
            )
        # If text isn't in the DB, insert date
        else:
            # Define insert statement
            self.cur.execute(
                """INSERT INTO quotes (text, tags, author) VALUES (?, ?, ?)""",
                (
                    item["text"],
                    str(item["tags"]),
                    item["author"]
                )
            )
            # Execute insert of data into database
            self.con.commit()

        return item
````

- このパイプラインを有効にするには、
  - `settings.py`を更新して、以前の`SqliteDemoPipeline`パイプラインではなく、`SqliteNoDuplicatesPipeline`を使用するようにする。

```python
# settings.py
ITEM_PIPELINES = {
    # "sqlite_demo.pipelines.SqliteDemoPipeline": 300,
    "sqlite_demo.pipelines.SqliteNoDuplicatesPipeline": 300,
}
```

これで、quotes spiderを実行すると、pipelineはデータベースにまだない新しいデータだけを保存するようになる。